# **ETAPA 4 - PROCESSAMENTO PARA CAMADA SILVER**

<br>

---

<br>

Essa etapa será responsável por mover os dados para a camada silver, filtrando e limpando os dados. 

> (Esse protótipo usa de referência fazer a desnormalização apenas na camada gold, mas caso necessário pode ser feito já nessa camada silver também)

<br>

*`Esse é um modelo genérico, coloque as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br> 

***AVISO IMPORTANTE: No Databricks Free Edition trabalha-se com catálogos, que é a maneira de na prática aplicar um Data Lakehouse***

<br><br>

---

<br>

### Parte 1 - **Importação das Bibliotecas Necessárias**

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

<br>

---

<br>

### Parte 2 - **Otimizar a Sessão com configurações Personalizadas**

Aqui o será configurado algumas propriedades para que o desempenho da sessão seja mais otimizado 
- Define tamanho fixo de partições para o shuffle para melhorar o paralelismo (usar ***número de partições = número de núclos de CPU * 2 ou 3*** para encontrar melhor cenário possível)
- Define o tamanho máximo de partições para evitar muitos arquivos pequenos
- Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita
- Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados

In [0]:
spark = (
    SparkSession.builder
        .appName("Load Data Bronze")
        .config("spark.sql.shuffle.partitions", "200")
        .config("spark.sql.files.maxPartitionBytes", "128MB")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .config("spark.sql.adaptive.enabled", "true")
        .getOrCreate()
)

<br>

---

<br>

### Parte 3 - **Definindo Origens e Destinos**

`Insira nas variáveis:` <br>
`--> nome_datalakehouse --> nome do Data Lakehouse ` <br>
`--> nome_camada_origem --> nome da camada de origem dos dados` <br>
`--> nome_volume_origem --> nome do volume de origem dos dados dentro da camada` <br>
`--> nome_camada_silver --> nome da camada de destino dos dados` <br>
`--> nome_volume_silver --> nome do volume de destino dos dados dentro da camada` <br>

In [0]:
nome_datalakehouse = "lhdw"
nome_camada_origem = "bronze"
nome_volume_origem = "vendas"
nome_camada_silver = "silver"
nome_volume_silver = "vendas"

O código a seguir armazena em variáveis os caminhos já prontos de origem e de destino dos dados:

In [0]:
origem_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_origem}/{nome_volume_origem}/"
silver_destino_dados = f"/Volumes/{nome_datalakehouse}/{nome_camada_silver}/{nome_volume_silver}/"

O código a seguir cria o volume de destino caso ele ainda não exista:

In [0]:
spark.sql(f"CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada_silver}.{nome_volume_silver}")

<br>

---

<br>

### Parte 4 - **Leitura dos Dados**


O código a seguir lê todos os dados e colocar em um Data Frame para que seja possível manipular os dados:

In [0]:
dataframe_dados = spark.read.format("parquet").load(origem_dados)

<br>

---

<br>

### Parte 5 - **Limpeza dos Dados**

A limpeza de dados é um processo crucial para garantir a **qualidade dos dados**. Isso envolve a remoção de **dados duplicados ou incorretos**, a **padronização de formatos e valores de dados** e o **enriquecimento de dados** com informações adicionais. Tudo isso para **garantir** que os **dados** sejam **precisos** e **confiáveis**.

Cada coluna deve ser analisada para encontrar possíveis inconformidades que podem ser encontradas e resolvidas nessa estapa, entre elas estão:
- Remoção de dados duplicados 
- 
- 
- 
- 

In [0]:
display(dataframe_dados)

In [0]:
dataframe_silver = (
    dataframe_dados.withColumn("Data", to_date(col("Data"), "yyyy-MM-dd")) 
        .withColumn("Email", lower(expr("regexp_replace(split(EmailNome, ':')[0], '[()]', '')"))) 
        .withColumn("Nome", expr("split(split(EmailNome, ':')[1], ', ')")) 
        .withColumn("Nome", expr("concat(Nome[1], ' ', Nome[0])")) 
        .withColumn("Cidade", expr("split(Cidade, ',')[0]")) 
        .withColumn("PrecoUnitario", format_number(col("PrecoUnitario"), 2)) 
        .withColumn("CustoUnitario", format_number(col("CustoUnitario"), 2)) 
        .withColumn("TotalVendas", format_number(col("PrecoUnitario").cast("double") * col("Unidades").cast("double"),2))
        .drop("EmailNome")
        .drop("IdCampanha")   
)
                     
display(dataframe_silver)


<br>

---

<br>

### Parte - ****

Texto

In [0]:
#

<br>

---

<br>

### **Resultado Final**

O código a seguir é uma das maneiras simples de se mostrar se deu certo ou não a importação do arquivo.


In [0]:
# Código está comentado por ser apenas uma demonstração. Para visualizar o resultado, retire o identificador de comentário e execute novamente

"""
 #
"""